In [1]:
import pandas as pd
import sqlite3
import pandas_profiling as pp

In [2]:
harmonized_table_file = "../../scoped-mapping/target/harmonized_table.db"
# created this manually.
#   see also env_package normailzastion in other repos...
#   but it alwyas seesm to boil down to a manual effort
package_targeting_file = "package_targeting.tsv"

ws_start = "('"
ws_end = "')"
ws_sep = "', '"

In [3]:
connection = sqlite3.connect(harmonized_table_file)

In [4]:
package_targeting = pd.read_csv(package_targeting_file, sep="\t")
package_targeting.fillna("", inplace=True)
package_targeting

,env_package,count,flag,repaired
0,host-associated,69981,,host-associated
1,human-gut,54078,True,human-gut
2,water,17924,,
3,built environment,14510,,
4,misc environment,14502,,
...,...,...,...,...
90,MIMARKS_16S,1,,
91,Deep-sea water,1,,
92,ocean[ENVO:00000015],1,,
93,marine sediment (ENVO:00002113),1,,


omitting "gut" at 172 and "gut microbiome" at 23 (compared to "human-gut" at 54078

In [5]:
raw_repaireds = list(set(package_targeting["repaired"]))
raw_repaireds

['',
 'human-skin',
 'human-vaginal',
 'human-associated',
 'human-gut',
 'gut',
 'host-associated',
 'human-oral']

In [6]:
valid_repaireds = [x for x in raw_repaireds if len(x) > 0]
valid_repaireds.sort()
valid_repaireds

['gut',
 'host-associated',
 'human-associated',
 'human-gut',
 'human-oral',
 'human-skin',
 'human-vaginal']

In [9]:
for i in valid_repaireds:
    print(i)

    target_subset = package_targeting.loc[
        package_targeting["flag"].eq(True) & package_targeting["repaired"].eq(i)
    ]
    if len(target_subset.index) > 0:
        where_string = ws_sep.join(list(target_subset["env_package"]))
        where_string = ws_start + where_string + ws_end
        where_string

        bshtq = (
            """select
            env_package,
            taxonomy_id,
            taxonomy_name,
            host_taxid,
            env_broad_scale,
            env_local_scale,
            env_medium,
            body_habitat,
            body_product,
            host,
            host_body_habitat,
            host_body_product,
            host_common_name,
            host_disease,
            host_sex,
            host_tissue_sampled,
            tissue
        from
            biosample b
        where
            --- both of the following will need to be tuned for each package
            env_package in """
            + where_string
            + """
            --- and taxonomy_name in ('human gut metagenome', 'Homo sapiens', 'human metagenome', 'gut metagenome')
            and (host_taxid = 9606 or host_taxid is null)
            --- or host_taxid is null or blank?"""
        )

        bshtres = pd.read_sql_query(bshtq, connection)

#         print(bshtres)

        bshtres = bshtres.astype(str)

        report_title = i + " Profiling Report"
        #         print(report_title)

        eda = pp.ProfileReport(
            bshtres,
            title=report_title,
            vars={
                "cat": {
                    "n_obs": 20,
                }
            }
        )

        profile_file = i + "_profile.html"
        #         print(profile_file)

        eda.to_file(profile_file)

gut


Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

...

human-vaginal


Summarize dataset:   0%|          | 0/30 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]